### Predict number of incoming calls (cagri_count)

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import VotingRegressor, StackingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor, Pool
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
train.drop(['BAŞLAMA_TARİHİ_VE_ZAMANI','SONA_ERME_TARİHİ_VE_ZAMANI','OutageID','KADEME','ŞEBEKE_UNSURU_KODU'], axis=1, inplace=True)
test.drop(['BAŞLAMA_TARİHİ_VE_ZAMANI','SONA_ERME_TARİHİ_VE_ZAMANI','OutageID','KADEME','ŞEBEKE_UNSURU_KODU'], axis=1, inplace=True)

In [4]:
cat_features = list(train.select_dtypes(object).columns)

In [5]:

for cat in cat_features:
    train[cat] = train[cat].replace(300,'bilinmeyen')
    le = LabelEncoder()
    le.fit(train[cat])
    train[cat] = le.transform(train[cat])
    test[cat] = le.transform(test[cat])

In [6]:
from sklearn.model_selection import cross_val_score
X = train.drop(['KESİNTİ_NO','cagri_count'], axis=1)
y = train['cagri_count']

In [7]:


estimators = [('lgbm', LGBMRegressor(verbose=-1)), 
             ('catboost', CatBoostRegressor(verbose=False)),
             ('xgboost', XGBRegressor())]

model = make_pipeline(StandardScaler(),  StackingRegressor(estimators=estimators))

cv_results = cross_val_score(model, X, y, cv=5, scoring='neg_root_mean_squared_error')

In [8]:
np.mean(cv_results)

-10.892016075501816